In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir y poner a prueba funciones definidas por el equipo QR para resolver un Sistema de Ecuaciones Lineales (SEL), a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    #Se inicializa una matriz llena de ceros con las dimensiones deseadas (mxn)
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            #Si entero es verdadero se obtiene el maximo entero menor o igual a 1 (//1)
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)

In [3]:
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v



In [5]:
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    #m contiene el numero de renglones y n el de columnas
    m=A.shape[0]
    n=A.shape[1]
    #se crea una matriz con los valores de A
    Arv=copy.copy(A)
    for j in range(n):
        beta, v=house(Arv[j:m,j])
        #Con esta multiplicación se van generando las componentes r de la matriz R
        Arv[j:m,j:n]=Arv[j:m,j:n]-beta*(np.outer(v,v)@Arv[j:m,j

In [7]:
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogonal y R es una matriz 

In [8]:
lines=inspect.getsource(fQR.matriz_R)
print(lines)

def matriz_R(A_r_v):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A, 
    apartir de la matriz Arv
    
    params: A_r_v   Matriz (mxn) que incluye elementos de la matriz R y Q, de la factorización QR

    return: R       Matriz (mxn) R de la factorización A=QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    R=np.zeros((m,n))
    #la matriz A_r_v ya tiene los elementos de r en el triangulo superior
    #por lo que únicamente se sustraen dichos valores y lo demás se deja en ceros
    for j in range(n):
        R[0:(j+1),j]=A_r_v[0:(j+1),j]
    return R



In [9]:
help(fQR.matriz_R)

Help on function matriz_R in module funciones_factorizacion_QR:

matriz_R(A_r_v)
    
    Descripción:
            Función que devuelve la matriz R de la factorización QR de una matriz A,
            apartir de la matriz Arv
    
    Parametros:
            A_r_v   np.ndarray que representa matriz (mxn) que incluye elementos de 
                    la matriz R y Q, de la factorización QR
    
    Resultado:
            R       np.ndarray que representa matriz (mxn) R de la factorización A=QR
    
    Visualmente la sustracción de los elementos de R se ve así:
         [ r11  r12  r13  r14 ]            [ r11  r12  r13  r14 ]
         [ v21  r22  r23  r24 ]            [  0   r22  r23  r24 ]
    Arv= [ v31  v32  r33  r34 ]   -->   R= [  0    0   r33  r34 ]
         [ v41  v42  v43  r44 ]            [  0    0    0   r44 ]
         [ v51  v52  v53  v54 ]            [  0    0    0    0  ]



In [10]:
lines=inspect.getsource(fQR.matriz_Q)
print(lines)

def matriz_Q(A_r_v):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A,
    apartir de la matriz Arv
    
    params: A_r_v   Matriz (mxn) con la info escencial para la factorización

    return: Q       Matriz Q (mxm) de la factorización A=QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Q=np.eye(m)
    I=np.eye(m)
    for j in range(n-1,-1,-1):
        #Se sustrae la información de los vectores de householder contenida
        #en la matriz Arv, agregando la primera entrada que no está en dicha matriz (y que es 1)
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        #Se calcula el factor beta para obtener la matriz de reflexión
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        #Aquí se va acumulando el producto de las Qj's para llegar a Q_(n-1)*Q(n-2)*...Q_2*Q_1*Q_0=Q
        #al final del ciclo
        Q[j:m,j:m]=(I[j:m,j:m]-beta*np.outer(v,v))@Q[j:m,j:m]
    #La Q_(n-1) es la matriz más chica y la última que cal

In [11]:
help(fQR.matriz_Q)

Help on function matriz_Q in module funciones_factorizacion_QR:

matriz_Q(A_r_v)
    
    Descripción:
            Función que devuelve la matriz R de la factorización QR de una matriz A,
            apartir de la matriz Arv
    
    Parametros:
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                    elementos de la matriz R y Q, de la factorización QR
    
    Resultado:
            R       np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    La matriz Q (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 
    vector de householder como: Q_j = Identidad(mxm) -

In [12]:
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            j       indica el índice de la matriz Q_j

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    #Para construir Q_j requerimos usar el vector v contenido en Arv contenido
    #en la j-1 columna (considerando que en Python la primer columna es la cero)
    v=np.concatenate((1,A_r_v[j:m,(j-1)]), axis=None)
    beta=2/(1+A_r_v[j:m,(j-1)].dot(A_r_v[j:m,(j-1)]))
    Qj[(j-1):m,(j-1):m]=np.eye(m-(j-1))-beta*np.outer(v,v)
    return Qj



In [13]:
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(A_r_v, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                    elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [14]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [15]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [16]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(7,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q=fQR.matriz_Q(Arv)
print('\n\nQ=\n',np.round(Q,4))
R=fQR.matriz_R(Arv)
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))



A=
 [[ 2.689  -5.1565 -3.1707 -4.343 ]
 [-4.2776  2.1528  1.9738  4.838 ]
 [-0.3533 -2.2015 -4.2322  3.5075]
 [ 1.2139 -4.7108  2.4052 -4.6841]
 [-5.8805 -1.8781  4.0793 -3.3311]
 [-3.5392 -2.3254 -3.3278 -4.2641]
 [ 1.0868  3.177  -2.9699  4.3436]]


Arv=
 [[ 8.6842 -0.609  -3.2233  0.0116]
 [ 0.7135  8.7704  0.8283  8.7947]
 [ 0.0589  0.5732  7.9439 -2.8119]
 [-0.2025  1.6697 -0.0442  6.2773]
 [ 0.9809 -0.7657 -0.3805 -0.2944]
 [ 0.5903 -0.1065  0.2529  0.353 ]
 [-0.1813 -0.6975  0.1707 -0.1736]]


Q=
 [[ 0.3096 -0.5664 -0.2144  0.0051 -0.5656 -0.439   0.1572]
 [-0.4926  0.2113  0.0266  0.4876 -0.598   0.2327  0.2501]
 [-0.0407 -0.2538 -0.5228  0.6803  0.3714 -0.0039 -0.2446]
 [ 0.1398 -0.5274  0.4145  0.1781  0.2413  0.4444  0.4929]
 [-0.6772 -0.2612  0.266  -0.0444  0.2581 -0.5704  0.0929]
 [-0.4075 -0.2934 -0.5537 -0.5154 -0.0151  0.4083  0.0927]
 [ 0.1251  0.3709 -0.3618  0.01    0.2439 -0.2521  0.7699]]


R=
 [[ 8.6842 -0.609  -3.2233  0.0116]
 [ 0.      8.7704  0.8283  8.7947

In [17]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[ 0.3096, -0.5664, -0.2144,  0.0051, -0.5656, -0.439 ,  0.1572],
       [-0.4926,  0.2113,  0.0266,  0.4876, -0.598 ,  0.2327,  0.2501],
       [-0.0407, -0.2538, -0.5228,  0.6803,  0.3714, -0.0039, -0.2446],
       [ 0.1398, -0.5274,  0.4145,  0.1781,  0.2413,  0.4444,  0.4929],
       [-0.6772, -0.2612,  0.266 , -0.0444,  0.2581, -0.5704,  0.0929],
       [-0.4075, -0.2934, -0.5537, -0.5154, -0.0151,  0.4083,  0.0927],
       [ 0.1251,  0.3709, -0.3618,  0.01  ,  0.2439, -0.2521,  0.7699]])

In [18]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1. -0.  0.  0.  0.  0. -0.]
 [-0.  1. -0.  0. -0.  0.  0.]
 [ 0. -0.  1. -0.  0. -0.  0.]
 [ 0.  0. -0.  1. -0.  0.  0.]
 [ 0. -0.  0. -0.  1. -0.  0.]
 [ 0.  0. -0.  0. -0.  1.  0.]
 [-0.  0.  0.  0.  0.  0.  1.]]


In [19]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,4,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\n',cadena,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))



A=
 [[ 1. -4.  3. -5.]
 [ 2. -5.  2.  0.]
 [-4.  2.  3. -3.]
 [ 0. -5.  1.  3.]
 [ 2.  5. -3. -3.]]

 Q1 :
 [[ 0.2  0.4 -0.8  0.   0.4]
 [ 0.4  0.8  0.4  0.  -0.2]
 [-0.8  0.4  0.2  0.   0.4]
 [ 0.   0.   0.   1.   0. ]
 [ 0.4 -0.2  0.4  0.   0.8]]

 Q1 @ A=
 [[ 5.  -2.4 -2.2  0.2]
 [-0.  -5.8  4.6 -2.6]
 [ 0.   3.6 -2.2  2.2]
 [ 0.  -5.   1.   3. ]
 [ 0.   4.2 -0.4 -5.6]]

 Q2 @ Q1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.614   0.3811 -0.5293  0.4446]
 [ 0.      0.3811  0.91    0.125  -0.105 ]
 [ 0.     -0.5293  0.125   0.8264  0.1458]
 [ 0.      0.4446 -0.105   0.1458  0.8775]]

 Q2 @ Q1 @ A=
 [[ 5.     -2.4    -2.2     0.2   ]
 [ 0.      9.4467 -4.3698 -1.6429]
 [ 0.      0.     -0.0821  1.974 ]
 [ 0.      0.     -1.9416  3.3139]
 [-0.     -0.      2.0709 -5.8637]]

 Q3 @ Q2 @ Q1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.     -0.0289 -0.6837  0.7292]
 [ 0.      0.     -0.6837  0.5457  0.4845]
 [ 0.      

In [20]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 6.46725908e+01,  1.31196816e+00, -7.69045232e+00, ...,
        -4.01742550e+00,  1.21980566e+00, -3.42835457e+00],
       [-1.32584193e-01,  7.17507016e+01,  2.10677654e+00, ...,
        -2.01733979e+01,  1.85817474e+00, -6.54984893e+00],
       [-1.26015116e-01, -2.17855368e-02,  6.23687593e+01, ...,
        -6.74197153e+00,  2.73531415e+00,  4.28121567e+00],
       ...,
       [-2.95607713e-02, -1.25254447e-01,  9.36207861e-02, ...,
         3.67259270e-01, -1.56431998e-01,  1.83502650e-02],
       [-2.30539956e-02,  9.64095557e-02,  8.65021096e-02, ...,
        -3.20188005e-01, -2.28578170e-01, -1.25792223e-01],
       [ 8.70862340e-03, -1.12782391e-01, -4.84944271e-02, ...,
        -2.02502122e-01,  1.48884761e-01,  1.79509307e-01]])

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [21]:
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    Arv=fQR.matriz_auxiliar_Arv(A)
    m=Arv.shape[0]
    n=Arv.shape[0]
    Q=fQR.matriz_Q(Arv)
    R=fQR.matriz_R(Arv)
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x



In [22]:
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas 
            con n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
                    elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)



## Eliminación por bloques

### Funciones

In [23]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [24]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        sys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [25]:
#Generamos un sistema de ecuaciones lineales
m=6
n=6
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-4.16  2.4  -2.92 -1.73  0.91  4.83]
 [ 5.78  1.03 -1.92 -5.79  5.38  2.94]
 [ 0.73 -4.34 -2.31 -2.67  3.89 -2.48]
 [-3.36  2.48  2.35 -0.79 -4.23  1.01]
 [ 1.96 -2.03  3.77 -5.54 -4.35  5.03]
 [ 0.09 -2.96 -4.08 -4.49 -4.79 -0.8 ]]
b: [[-1.84]
 [-3.12]
 [-0.08]
 [-0.61]
 [-3.02]
 [-2.38]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [26]:
np.linalg.solve(A,b)

array([[-0.08182894],
       [-0.10585086],
       [ 0.14062215],
       [ 0.39190967],
       [ 0.10589978],
       [-0.19339849]])

Ahora usemos la factorización QR

In [27]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

NameError: name 'fQR' is not defined

In [ ]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

Por último usemos eliminación por bloques con QR

In [ ]:
eliminacion_bloques(A,b)

### B) Sistema sin solución

In [ ]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [5]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [ ]:
x=Solucion_SEL_QR_nxn(A,b)
x

In [ ]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

Por último usemos eliminación por bloques con QR

In [ ]:
eliminacion_bloques(A,b)

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [ ]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [0]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [ ]:
x=Solucion_SEL_QR_nxn(A,b)
x

In [ ]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

Por último usemos eliminación por bloques con QR

In [ ]:
eliminacion_bloques(A,b)